# AFTER INSERT trigger

In PostgreSQL, a trigger is a database object associated with a table, which is automatically fired in response to an `INSERT`, `UPDATE`, `DELETE`, or `TRUNCATE` event.
* An `AFTER INSERT` trigger is a trigger that is fired after an `INSERT` event occurs on a table.
* The `AFTER INSERT` trigger can access the newly inserted data using the `NEW` record variable.
* This `NEW` variable allows you to access the values of columns in the inserted row as `NEW.column_name`.
* Typically, you use `AFTER INSERT` triggers for logging changes, updating related tables, or sending notifications based on the inserted data.

To create an `AFTER INSERT` trigger, you follow these steps:

---

**First, define a function that will execute when the trigger is activated**:

```postgresql
CREATE OR REPLACE FUNCTION trigger_function()
   RETURNS TRIGGER
   LANGUAGE PLPGSQL
AS
$$
BEGIN
   -- trigger logic
   -- ...
   RETURN NEW;
END;
$$
```

The `RETURN NEW` statement indicates that the function returns the modified row, which is the `NEW` row.

---

**Second, create an `AFTER INSERT` trigger and bind the function to it**:

```postgresql
CREATE TRIGGER trigger_name
AFTER INSERT
ON table_name
FOR EACH {ROW | STATEMENT}
EXECUTE FUNCTION trigger_function(); 
```

# AFTER INSERT trigger example

**First, create a new table called `members` to store the member data**:

```postgresql
CREATE TABLE members (
    id SERIAL PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    email VARCHAR(100) UNIQUE
);
```

**Second, create another table called `memberships` to store the memberships of the members**:

```postgresql
CREATE TABLE memberships (
    id SERIAL PRIMARY KEY,
    member_id INT NOT NULL REFERENCES members(id),
    membership_type VARCHAR(50) NOT NULL DEFAULT 'free'
); 
```

**Third, define a trigger function that inserts a default free membership for every member**:

```postgresql
CREATE OR REPLACE FUNCTION create_membership_after_insert()
    RETURNS TRIGGER
    LANGUAGE plpgsql
AS 
$$
BEGIN
    INSERT INTO memberships (member_id)
    VALUES (NEW.id);
    RETURN NEW;
END;
$$ 
```

**Fourth, define an `AFTER INSERT` trigger on the `members` table, specifying that it should execute the `create_membership_after_insert()` function for each row inserted**:

```postgresql
CREATE TRIGGER after_insert_member_trigger
AFTER INSERT ON members
FOR EACH ROW
EXECUTE FUNCTION create_membership_after_insert(); 
```

**Fifth, insert a new row into the `members` table**:

```postgresql
INSERT INTO members(name, email)
VALUES('John Doe', 'john.doe@example.com')
RETURNING *; 
```

**Sixth, retrieve data from the `memberships` table**:

```postgresql
SELECT * FROM memberships; 
```